In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
#ADD YOUR DIRECTORY HERE
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/utils')
sys.path.append('/Users/lorenzostigliano/Documents/University/Imperial/Summer Term/thesis-imperial/src/models')

from utils.analysis import *
from utils.getters import * 
from utils.plotters import * 
from utils.helpers import * 
from utils.config import * 
from models_config.model_config_GSP import *

/var/folders/1j/5s36wz6s0jlb1k64vsq1k2zh0000gn/T/ipykernel_95922/2915192044.py:17: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]


## Getting the best model for RepKD

In [2]:
analysis_type="model_assessment"
dataset_split="val"
dataset="gender_data"
models_args=[
    gcn_student_lsp_ensamble_2_args,
    gcn_student_lsp_ensamble_3_args,
    gcn_student_lsp_ensamble_4_args,
    gcn_student_lsp_ensamble_5_args
]
views = [0,2,4,5]

#GET ALL OF THE MODEL BEST STUDENTS FOR ALL RUNS AND DIFFERENT PICKING CRITERIA FOR OUR MODEL 
all_model_metrics_mean, all_model_metrics_var, all_model_best_student = get_all_best_student(analysis_type, dataset_split, dataset, models_args, views)

gcn_student_lsp_ensamble_2
[[0.9384722222222223, 0.00025889274691358056, 1], [0.9383333333333332, 0.0003453703703703699, 0], [0.9384722222222223, 0.00025889274691358056, 1], [0.9383333333333332, 0.0003453703703703699, 0], [0.9384722222222223, 0.00025889274691358056, 1]]
[[0.9444444444444444, 0.00037461419753086217, 0], [0.8927777777777777, 0.0002483796296296277, 1], [0.8927777777777777, 0.0002483796296296277, 1], [0.9444444444444444, 0.00037461419753086217, 0], [0.8927777777777777, 0.0002483796296296277, 1]]
[[0.9579166666666667, 0.00031059027777777855, 1], [0.9579166666666667, 0.00031059027777777855, 1], [0.9579166666666667, 0.00031059027777777855, 1], [0.9579166666666667, 0.00031059027777777855, 1], [0.9579166666666667, 0.00031059027777777855, 1]]
[[0.9577777777777777, 0.0002234567901234572, 0], [0.9330555555555555, 0.0003340277777777791, 1], [0.9330555555555555, 0.0003340277777777791, 1], [0.9330555555555555, 0.0003340277777777791, 1], [0.9330555555555555, 0.0003340277777777791, 1]]

In [4]:
df_rep, df_acuracy, df_var, index = final_student(all_model_metrics_mean, all_model_metrics_var, all_model_best_student, selection_method='weighted acc')

In [5]:
df_rep, df_acuracy, df_var, index

(          2         3         4         5
 0  0.938333  0.968333  0.960000  0.958056
 1  0.944444  0.931944  0.968889  0.978333
 2  0.957917  0.966667  0.965000  0.977222
 3  0.933056  0.921667  0.969028  0.933750
 4  0.943437  0.947153  0.965729  0.961840,
           2         3         4         5
 0  0.563876  0.611339  0.601303  0.589203
 1  0.600378  0.601620  0.619532  0.614281
 2  0.653475  0.641890  0.640231  0.638833
 3  0.633602  0.634708  0.631901  0.617351
 4  0.612833  0.622390  0.623242  0.614917,
           2         3         4         5
 0  0.004213  0.005999  0.009602  0.007251
 1  0.003084  0.007322  0.004146  0.002374
 2  0.005716  0.002912  0.002405  0.002883
 3  0.003754  0.005351  0.003948  0.005817
 4  0.004302  0.005628  0.005718  0.005009,
      2    3    4    5
 0  0.0  2.0  2.0  2.0
 1  0.0  2.0  1.0  1.0
 2  1.0  2.0  2.0  2.0
 3  1.0  1.0  1.0  1.0)

## COMPARISON WITH OTHER METHODS

In [6]:
#GET THE REPRODUCIBILITY SCORE FOR ALL MODELS 
mean_all_runs, mean_all_std = reproducibility_mulitple_runs_student_specific(
    dataset="gender_data", 
    views=[0, 2, 4, 5], 
    models=[  
        "gcn", 
        "gcn_student",
        "gcn_student_teacher",
        "fitnet", 
        "lsp", 
        "mskd",
        "gcn_student_lsp_ensamble_4" 
    ], 
    CV=["3Fold", "5Fold", "10Fold"], 
    runs=[i for i in range(10)], 
    students=[
        [2,1,2,1]
    ], 
    model_args= [
        gcn_args,
        gcn_student_args,
        gcn_student_args, 
        gcn_fitnet_student_args, 
        gcn_lsp_student_args, 
        gcn_mskd_student_args,
        gcn_student_lsp_ensamble_4_args
    ])

In [10]:
reproducibility_frame = pd.DataFrame(mean_all_runs.T)

In [11]:
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet_gcn", "lsp_gcn", "mskd_gcn", "repKD"]
reproducibility_frame.columns = models
reproducibility_frame = reproducibility_frame*100
reproducibility_frame["delta student"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gcn"]) / reproducibility_frame["gcn"])*100
reproducibility_frame["delta teacher"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gcn_student"]) / reproducibility_frame["gcn_student"])*100

reproducibility_frame["delta vanilla"] = ((reproducibility_frame["repKD"] - reproducibility_frame["gcn_student_teacher"]) / reproducibility_frame["gcn_student_teacher"])*100
reproducibility_frame["delta fitnet"] = ((reproducibility_frame["repKD"] - reproducibility_frame["fitnet_gcn"]) / reproducibility_frame["fitnet_gcn"])*100
reproducibility_frame["delta lsp"] = ((reproducibility_frame["repKD"] - reproducibility_frame["lsp_gcn"]) / reproducibility_frame["lsp_gcn"])*100
reproducibility_frame["delta mskd"] = ((reproducibility_frame["repKD"] - reproducibility_frame["mskd_gcn"]) / reproducibility_frame["mskd_gcn"])*100

reproducibility_frame.T

,0,1,2,3,4
gcn,89.138889,88.875000,90.597222,90.166667,89.694444
gcn_student,92.597222,94.041667,91.166667,93.291667,92.774306
gcn_student_teacher,90.069444,92.847222,90.569444,92.583333,91.517361
fitnet_gcn,63.500000,70.277778,67.861111,73.930556,68.892361
lsp_gcn,70.486111,77.305556,74.402778,77.916667,75.027778
mskd_gcn,82.444444,81.847222,78.611111,79.458333,80.590278
repKD,96.000000,96.888889,96.500000,96.902778,96.572917
delta student,7.697102,9.017034,6.515407,7.470733,7.668783
delta teacher,3.674816,3.027618,5.850091,3.870776,4.094465
delta vanilla,6.584426,4.353029,6.548075,4.665467,5.524149


In [12]:
reproducibility_frame_std = pd.DataFrame(mean_all_std.T)*100
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet_gcn", "lsp_gcn", "mskd_gcn", "repKD"]
reproducibility_frame_std.columns = models
reproducibility_frame_std.T

,0,1,2,3,4
gcn,3.737531,3.388120,4.468274,5.456336,3.138927
gcn_student,4.644811,2.196421,3.407848,2.696055,1.628206
gcn_student_teacher,4.762593,3.530210,5.428566,2.627314,2.296767
fitnet_gcn,8.245088,11.256000,10.113019,9.168571,6.401246
lsp_gcn,11.524975,8.329444,12.748767,7.741483,4.074999
mskd_gcn,4.523686,7.274398,9.169402,11.048338,5.176057
repKD,1.917270,0.771302,1.105542,1.394503,0.751646


In [13]:
views=[0, 2, 4, 5]
CV=["3Fold", "5Fold", "10Fold"]

runs = [i for i in range(10)]
dataset = "gender_data"
dataset_split="val"
analysis_type="model_assessment"
metrics = ["acc"]

final_model = ["gcn_student_lsp_ensamble_4"]
final_model_args = [gcn_student_lsp_ensamble_4_args]
best_student = [2,1,2,1]
baseline_models = ["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd"]
baseline_models_args = [
    gcn_args,
    gcn_student_args,
    gcn_student_args, 
    gcn_fitnet_student_args, 
    gcn_lsp_student_args, 
    gcn_mskd_student_args
]

In [23]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics)

In [24]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100

average_across_views_df["delta teacher"] = ((average_across_views_df["repKD"] - average_across_views_df["gcn"]) / average_across_views_df["gcn"])*100
average_across_views_df["delta student"] = ((average_across_views_df["repKD"] - average_across_views_df["gcn_student"]) / average_across_views_df["gcn_student"])*100
average_across_views_df["delta vanilla"] = ((average_across_views_df["repKD"] - average_across_views_df["gcn_student_teacher"]) / average_across_views_df["gcn_student_teacher"])*100
average_across_views_df["delta fitnet"] = ((average_across_views_df["repKD"] - average_across_views_df["fitnet"]) / average_across_views_df["fitnet"])*100
average_across_views_df["delta lsp"] = ((average_across_views_df["repKD"] - average_across_views_df["lsp"]) / average_across_views_df["lsp"])*100
average_across_views_df["delta mskd"] = ((average_across_views_df["repKD"] - average_across_views_df["mskd"]) / average_across_views_df["mskd"])*100

average_across_views_df.T

,0,1,2,3,4
gcn,61.010917,66.158171,66.906934,65.441907,64.879482
gcn_student,57.784444,63.747176,63.248662,63.622665,62.100737
gcn_student_teacher,58.161648,63.981767,63.754955,63.765708,62.416019
fitnet,60.725618,64.208859,65.195896,64.446336,63.644177
lsp,61.209564,65.579805,66.425911,65.784582,64.749965
mskd,62.182683,65.721309,66.507907,65.786443,65.049585
repKD,60.130316,61.953215,64.023050,63.190064,62.324161
delta teacher,-1.443350,-6.355913,-4.310292,-3.440980,-3.938565
delta student,4.059694,-2.814180,1.224355,-0.679948,0.359778
delta vanilla,3.384822,-3.170514,0.420509,-0.902749,-0.147170


In [26]:
average_across_views_df_std_mean = pd.DataFrame(average_across_views_df_std_mean)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df_std_mean.columns = models
average_across_views_df_std_mean.T*100

,0,1,2,3
gcn,0.297329,0.092750,0.087278,0.124343
gcn_student,0.279307,0.198404,0.111084,0.086607
gcn_student_teacher,0.239705,0.135317,0.147560,0.228624
fitnet,0.573669,0.392149,0.308234,0.200388
lsp,0.267381,0.308585,0.082510,0.130457
mskd,0.142362,0.382171,0.263090,0.085180
repKD,0.185916,0.051010,0.057718,0.195582


## F1 Score, recall, precision

In [52]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics=["f1"])

In [53]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100
average_across_views_df.T

,0,1,2,3,4
gcn,51.735460,58.766552,59.251278,58.912915,57.166551
gcn_student,40.648281,53.587911,51.921626,54.380161,50.134495
gcn_student_teacher,41.150978,54.788663,53.297530,54.823061,51.015058
fitnet,49.461677,55.754699,57.191555,57.447045,54.963744
lsp,53.364957,59.131922,59.792522,59.784446,58.018462
mskd,53.151701,57.996565,58.839393,58.544888,57.133137
repKD,58.619063,57.488726,60.286300,58.523134,58.729306


In [54]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics=["recall"])

In [55]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100
average_across_views_df.T

,0,1,2,3,4
gcn,48.996144,55.773050,55.945532,57.891319,54.651511
gcn_student,34.117635,48.096669,46.099689,49.800897,44.528723
gcn_student_teacher,34.603360,50.022187,47.937657,50.697767,45.815243
fitnet,45.501669,52.236012,53.447631,55.409159,51.648618
lsp,51.085302,57.035675,57.252301,58.573164,55.986610
mskd,49.571056,54.552392,55.060994,56.443901,53.907086
repKD,45.495810,54.808317,56.893666,57.158208,53.589000


In [56]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics=["precision"])

In [57]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100
average_across_views_df.T

,0,1,2,3,4
gcn,57.403173,63.757823,65.431722,62.198965,62.197921
gcn_student,53.234345,61.292470,60.892395,60.665802,59.021253
gcn_student_teacher,54.166348,61.174658,61.401621,60.634130,59.344189
fitnet,56.240577,60.993275,62.757585,60.914219,60.226414
lsp,56.793873,62.285688,63.505454,62.039190,61.156051
mskd,58.486140,63.221057,64.560608,62.412132,62.169984
repKD,39.249990,52.955959,54.486966,56.595529,50.822111
